In [1]:

import sys
import os
from csbdeep.utils import Path, normalize
import matplotlib.pyplot as plt
from skimage.morphology import remove_small_objects
import glob
import cv2
from scipy.ndimage.morphology import binary_fill_holes
from tifffile import imread, imwrite
from csbdeep.utils import Path, normalize
from Utils.helpers import multiplot, doubleplot, WingArea
from csbdeep.models import Config, CARE
from skimage.filters import threshold_otsu


Using TensorFlow backend.


In [ ]:
Masterdir =  '/media/sancere/Images_For_FAQT/'

MaskResults = Masterdir + '/MaskResults/'

AreaResults = Masterdir + '/AreaResults/'
AreaResultsName = 'Area'

Model_Dir = '/home/sancere/Kepler/SegmentationModel/'

UNETSegmentationModelName = 'DeepWingSegmentationModelUNET'


UnetModel = CARE(config = None, name = UNETSegmentationModelName, basedir = Model_Dir)

In [ ]:
#Apply the prediction on left wing directory
min_size = 15000
Raw_path = os.path.join(Masterdir, '*tif')
Path(MaskResults).mkdir(exist_ok = True)
Path(AreaResults).mkdir(exist_ok = True)
axes = 'YXC'
saveaxes = 'YX'
filesRaw = glob.glob(Raw_path)
filesRaw.sort
count = 0
axis_norm = (0,1,2)   # normalize channels independently

AllArea = []
AllName = []
for fname in filesRaw:
          
            #Read image        
            image = imread(fname)
            originalX = image.shape[0]
            originalY = image.shape[1]  
            
            Name = os.path.basename(os.path.splitext(fname)[0])
         
            x = image

            Segmented = UnetModel.predict(x,axes)
            thresh = threshold_otsu(Segmented) 
            Binary = Segmented > thresh
            Filled = binary_fill_holes(Binary)
            NoSmall = remove_small_objects(Filled, min_size)
            Finalimage = CCLabels(NoSmall)
            RightArea, LeftArea, RightMinusLeft, RightPlusLeft, Assymetery = WingArea(Finalimage, Finalimage)
            AllArea.append(RightArea)
            AllName.append(Name)
            
            doubleplot(image,Finalimage[:,:,0], 'Original', 'UNET', plotTitle = 'Segmentation Result' )

            imwrite((MaskResultsRight + 'Mask' + Name + '.tif' ) , Finalimage.astype('uint8'))
            
df = pd.DataFrame(list(zip(AllArea)), index =AllName, 
                                              columns =['Area'])

df.to_csv(AreaResults + '/' + AreaResultsName +  '.csv')  
df            